In [60]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl

In [61]:
input_file = "Crimes.csv"

In [62]:
df = pd.read_csv(input_file, skip_blank_lines=True)
keep_col = ['Date','Block','Primary Type','Description','Arrest','District','Ward','FBI Code', 'Latitude', 'Longitude']
new_df = df[keep_col]

In [63]:
new_df.head()

,Date,Block,Primary Type,Description,Arrest,District,Ward,FBI Code,Latitude,Longitude
0,05/03/2016 09:00:00 PM,042XX W MADISON ST,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,True,11,28,18,41.880658,-87.731212
1,05/03/2016 09:35:00 PM,065XX S KEDVALE AVE,THEFT,OVER $500,False,8,13,06,41.774141,-87.726307
2,05/03/2016 02:45:00 PM,045XX S DAMEN AVE,ASSAULT,SIMPLE,False,9,12,08A,41.811698,-87.675214
3,05/03/2016 06:42:00 PM,063XX S CLAREMONT AVE,BATTERY,DOMESTIC BATTERY SIMPLE,False,8,15,08B,41.778661,-87.682273
4,05/03/2016 03:00:00 PM,070XX S INDIANA AVE,ROBBERY,STRONGARM - NO WEAPON,False,3,6,03,41.766961,-87.620152


In [64]:
new_df.insert(10,'Safety Value','')
new_df.insert(11,'Label','')
#df[~df.Block.str.contains("كونت")]

In [65]:
new_df.head()

,Date,Block,Primary Type,Description,Arrest,District,Ward,FBI Code,Latitude,Longitude,Safety Value,Label
0,05/03/2016 09:00:00 PM,042XX W MADISON ST,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,True,11,28,18,41.880658,-87.731212,,
1,05/03/2016 09:35:00 PM,065XX S KEDVALE AVE,THEFT,OVER $500,False,8,13,06,41.774141,-87.726307,,
2,05/03/2016 02:45:00 PM,045XX S DAMEN AVE,ASSAULT,SIMPLE,False,9,12,08A,41.811698,-87.675214,,
3,05/03/2016 06:42:00 PM,063XX S CLAREMONT AVE,BATTERY,DOMESTIC BATTERY SIMPLE,False,8,15,08B,41.778661,-87.682273,,
4,05/03/2016 03:00:00 PM,070XX S INDIANA AVE,ROBBERY,STRONGARM - NO WEAPON,False,3,6,03,41.766961,-87.620152,,


In [66]:
new_df.insert(3,'Crime Weightage','')
new_df.head()
crime_type = new_df['Primary Type']
crime_weightage = new_df['Crime Weightage']

In [67]:
#crime_weightage[1] = 4
for i in crime_type:
    #print(i)
    if(i == 'NARCOTICS' or i == 'ASSAULT' or i == 'SEX OFFENSE' or 'CRIM SEXUAL ASSAULT' or 'HOMICIDE' or 'HUMAN TRAFFICKING'):
        print(i)
        #crime_weightage[j] = 4
    #j = j+1
new_df.head()        
#print('hello')   

NARCOTICS
ASSAULT
ASSAULT
NARCOTICS
ASSAULT
NARCOTICS
ASSAULT
NARCOTICS
ASSAULT
ASSAULT
ASSAULT
ASSAULT
SEX OFFENSE
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
NARCOTICS
SEX OFFENSE
ASSAULT
NARCOTICS
NARCOTICS
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
NARCOTICS
NARCOTICS
ASSAULT
NARCOTICS
ASSAULT
NARCOTICS
NARCOTICS
ASSAULT
NARCOTICS
NARCOTICS
NARCOTICS
NARCOTICS
ASSAULT
NARCOTICS
ASSAULT
NARCOTICS
NARCOTICS
NARCOTICS
ASSAULT
NARCOTICS
NARCOTICS
NARCOTICS
NARCOTICS
NARCOTICS
NARCOTICS
ASSAULT
NARCOTICS
NARCOTICS
NARCOTICS
ASSAULT
ASSAULT
ASSAULT
NARCOTICS
NARCOTICS
NARCOTICS
ASSAULT
ASSAULT
ASSAULT
NARCOTICS
NARCOTICS
NARCOTICS
ASSAULT
NARCOTICS
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
NARCOTICS
NARCOTICS
ASSAULT
ASSAULT
NARCOTICS
ASSAULT
NARCOTICS
ASSAULT
ASSAULT
NARCOTICS
ASSAULT
ASSAULT
ASSAULT
NARCOTICS
NARCOTICS
NARCOTICS
ASSAULT
NARCOTICS
NARCOTICS
NARCOTICS
ASSAULT
ASSAULT
ASSAULT
ASSAULT
ASSAULT
SEX OFFENSE
NARCOTICS
ASSAULT
NARC

,Date,Block,Primary Type,Crime Weightage,Description,Arrest,District,Ward,FBI Code,Latitude,Longitude,Safety Value,Label
0,05/03/2016 09:00:00 PM,042XX W MADISON ST,NARCOTICS,,POSS: CANNABIS 30GMS OR LESS,True,11,28,18,41.880658,-87.731212,,
1,05/03/2016 09:35:00 PM,065XX S KEDVALE AVE,THEFT,,OVER $500,False,8,13,06,41.774141,-87.726307,,
2,05/03/2016 02:45:00 PM,045XX S DAMEN AVE,ASSAULT,,SIMPLE,False,9,12,08A,41.811698,-87.675214,,
3,05/03/2016 06:42:00 PM,063XX S CLAREMONT AVE,BATTERY,,DOMESTIC BATTERY SIMPLE,False,8,15,08B,41.778661,-87.682273,,
4,05/03/2016 03:00:00 PM,070XX S INDIANA AVE,ROBBERY,,STRONGARM - NO WEAPON,False,3,6,03,41.766961,-87.620152,,
